In [159]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl,
    basemaps
)

from traitlets import link

In [160]:
center = [-12.543839666237682, 131.08474731445315]
zoom = 10


m = Map(center=center, zoom=zoom,layout=dict(width='400px', height='400px'))


In [161]:
#Let's draw a second map and try to import this GeoJSON data into it.

m2 = Map(center=center, zoom=zoom, basemap=basemaps.Esri.WorldImagery,layout=dict(width='400px', height='400px'))


In [162]:
#Now create the DrawControls and add it to the Maps using add_control. We also register a handler for draw events. This will fire when a drawn path is created, edited or deleted (there are the actions). The geo_json argument is the serialized geometry of the drawn path, along with its embedded style.

draw_control = DrawControl(polyline={})
draw_control2 = DrawControl(polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={})
def handle_draw(self, action, geo_json):
    m2.add_layer(GeoJSON(data=draw_control.last_draw))
    draw_control2.last_draw =draw_control.last_draw
    
def handle_draw2(self, action, geo_json):
    m.add_layer(GeoJSON(data=draw_control2.last_draw))
    draw_control.last_draw =draw_control2.last_draw
   
    
#add handlers to draw controls  
draw_control.on_draw(handle_draw)
draw_control.on_draw(handle_draw)
draw_control2.on_draw(handle_draw2)

#add draw controls to maps
m.add_control(draw_control)
m2.add_control(draw_control2)

In [163]:
#We can use link to synchronize traitlets of the two maps:

map_center_link = link((m, 'center'), (m2, 'center'))
map_zoom_link = link((m, 'zoom'), (m2, 'zoom'))

new_poly = GeoJSON(data=draw_control.last_draw)

m2.add_layer(new_poly)

In [164]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[-12.543839666237682, 131.08474731445315], controls=(DrawControl(layer=FeatureGroup(), polygon={'shapeOptions': {}}),), default_tiles=TileLayer(max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), layers=(TileLayer(max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']),), layout=Layout(height='400px', width='400px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=10)

In [165]:
m2

Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', 'max_zoom': 20, 'attribution': 'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community', 'name': 'Esri.WorldImagery'}, center=[-12.543839666237682, 131.08474731445315], controls=(DrawControl(layer=FeatureGroup(), polygon={'shapeOptions': {'color': '#0000FF'}}),), default_tiles=TileLayer(attribution='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community', max_zoom=20, min_zoom=1, name='Esri.WorldImagery', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size'], url='http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'), layers=(TileLayer(attribution='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community', max_zoom=20, min_zoom=1, name='Esri.WorldImagery', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size'], url='http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'), GeoJSON(data={'type': 'Feature', 'geometry': None})), layout=Layout(height='400px', width='400px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=10)

In [167]:
#now test if we can query using the poly to load data 
#TODO fix this so it uses the correct projection

import datacube
from shapely.geometry import Polygon
from datacube.utils import geometry
dc=datacube.Datacube()
#TODO need to project poly to get area
#Leaflet uses Web Mercator EPSG:3857

# geom_crs = geometry.CRS('EPSG:3857')
geom_crs = geometry.CRS('EPSG:4326') 
geom=geometry.Geometry(draw_control.last_draw['geometry'], geom_crs)
#poly = Polygon(draw_control.last_draw['geometry']['coordinates'][0])
print(geom)

start_of_epoch = '2011-01-01'
end_of_epoch =  '2011-12-31'

# query = {
#         'time': (start_of_epoch, end_of_epoch), 'geopolygon': geom
#     }
#Define wavelengths/bands of interest, remove this kwarg to retrieve all bands
bands_of_interest = ['blue',
                         'green',
                         'red',
                         'nir',
                         'swir1',
                         'swir2'
                         ]

query = {
        'time': (start_of_epoch, end_of_epoch), 'geopolygon': geom
    }

data= dc.load(product= 'ls7_nbart_albers', measurements=bands_of_interest,
                 group_by='solar_day', **query)
data

Geometry({'type': 'Polygon', 'coordinates': [[(130.99383, -12.339457), (131.009451, -12.340463), (131.006189, -12.363269), (130.9928, -12.364107), (130.99383, -12.339457)]]}, CRS('EPSG:4326'))


<xarray.Dataset>
Dimensions:  (time: 19, x: 74, y: 108)
Coordinates:
  * time     (time) datetime64[ns] 2011-02-01T01:16:10.500000 ...
  * y        (y) float64 -1.302e+06 -1.302e+06 -1.302e+06 -1.302e+06 ...
  * x        (x) float64 -1.115e+05 -1.115e+05 -1.115e+05 -1.115e+05 ...
Data variables:
    blue     (time, y, x) int16 841 841 824 824 807 790 790 807 824 876 876 ...
    green    (time, y, x) int16 1145 1200 1089 1071 1071 1071 1052 1071 1126 ...
    red      (time, y, x) int16 1197 1181 1165 1148 1165 1148 1148 1181 1214 ...
    nir      (time, y, x) int16 1434 1474 1434 1393 1393 1353 1353 1353 1434 ...
    swir1    (time, y, x) int16 312 289 266 289 562 608 539 517 585 722 517 ...
    swir2    (time, y, x) int16 170 145 120 170 269 294 294 244 269 343 269 ...
Attributes:
    crs:      EPSG:3577